In [0]:
pip install indic-nlp-library

In [0]:
pip install torchtext==0.5

     |████████████████████████████████| 81kB 2.8MB/s 
     |████████████████████████████████| 1.0MB 9.6MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [0]:
import codecs
import torch
import torch.nn as nn
import random
import math
import numpy as np

In [0]:
SEED = 21

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
with codecs.open('/content/drive/My Drive/Colab Notebooks/traing_english.txt', encoding='utf-8') as f:
  training_english = f.read().split('\n')

In [0]:
with codecs.open('/content/drive/My Drive/Colab Notebooks/traing_hindi.txt', encoding='utf-8') as f:
  training_hindi = f.read().split('\n')

In [0]:
with codecs.open('/content/drive/My Drive/Colab Notebooks/testing_english.txt', encoding='utf-8') as f:
  testing_english = f.read().split('\n')

In [0]:
with codecs.open('/content/drive/My Drive/Colab Notebooks/testing_hindi.txt', encoding='utf-8') as f:
  testing_hindi = f.read().split('\n')

In [0]:
from indicnlp.tokenize import indic_tokenize  
from nltk.tokenize import RegexpTokenizer

In [0]:
def tokenizer(hindi, english, flag):
  data = []
  corpus_hindi = ['<sos>','<eos>','<unk>','<pad>']
  corpus_english = ['<sos>','<eos>','<unk>','<pad>']
  tokenizer_english = RegexpTokenizer('\w+|\$[\.]+|\S+')
  for sentence_hindi, sentence_english in zip(hindi, english):
    dic = {}
    h = indic_tokenize.trivial_tokenize(sentence_hindi)
    e = tokenizer_english.tokenize(sentence_english)
    if flag:
      corpus_hindi += h
      corpus_english += e
    dic['src'] = e
    dic['trg'] = h
    data.append(dic)
  return data, corpus_hindi, corpus_english

In [0]:
training_data, corpus_hindi, corpus_english = tokenizer(training_hindi,training_english, True)
training_data[0]

{'src': ['dwarka',
  'is',
  'also',
  'a',
  'dhaam',
  'among',
  'the',
  'chaar',
  'dhaams'],
 'trg': ['द्वारका', 'चार', 'धामों', 'में', 'एक', 'धाम', 'भी', 'है']}

In [0]:
type(training_data)

list

In [0]:
# testing_data, _, _ = tokenizer(testing_hindi, testing_english, False)
# testing_data[0]

In [0]:
valid_test_data, _, _ = tokenizer(testing_hindi, testing_english, False)
valid_test_data[0]

{'src': ['the',
  'place',
  'baidadih',
  'is',
  'famous',
  'for',
  'the',
  'big',
  'well'],
 'trg': ['बैदाडीह', 'स्थान', 'बड़े', 'कुएँ', 'के', 'लिए', 'प्रसिद्ध', 'है']}

In [0]:
valid_test_data[15000]

{'src': ['only', 'one', 'family', 'stays', 'over', 'here'],
 'trg': ['यहाँ', 'अब', 'एक', 'ही', 'परिवार', 'रहता', 'है']}

In [0]:
valid_data = valid_test_data[:len(valid_test_data)//2]
testing_data = valid_test_data[len(valid_test_data)//2:]
print(len(valid_data))

9980


In [0]:
print(len(testing_data))

9981


In [0]:
testing_data[5219] #Test example

{'src': ['first',
  'of',
  'all',
  'asia',
  'is',
  'divided',
  'into',
  'sea',
  'and',
  'rivers'],
 'trg': ['सर्वप्रथम',
  'एशिया',
  'समुद्रों',
  'और',
  'नदियों',
  'में',
  'बँटा',
  'हुआ',
  'है']}

In [0]:
print(testing_english[15000]) #Test example
print(testing_hindi[15000])

only one family stays over here
यहाँ अब एक ही परिवार रहता है


In [0]:
from collections import Counter
counter_english = Counter(corpus_english)

In [0]:
vocab_hindi = {}
count = 0
for i,word in enumerate(corpus_hindi):
  if word not in vocab_hindi:
    vocab_hindi[word] = count
    count += 1

In [0]:
#vocab_hindi

In [0]:
vocab_english = {'<sos>': 0,'<eos>': 1,'<unk>': 2,'<pad>':3}
count = 4
for i,word in enumerate(corpus_english):
  if word not in vocab_english and counter_english[word] > 1:
    vocab_english[word] = count
    count += 1

In [0]:
vocab_english["only"]

221

In [0]:
# for i in vocab_hindi:
#   print(vocab_hindi[i])

In [0]:
print(len(vocab_english), len(vocab_hindi))

13445 25598


In [0]:
#training_data[:2]

In [0]:
def vectorization(data):
  for item in data:
    wordvec_english = []
    wordvec_hindi = []
    for word in item['src']:
      if word not in vocab_english:
        word = '<unk>'
      wordvec_english.append(vocab_english[word])
    item['src'] = wordvec_english[::-1]
    
    for word in item['trg']:
      if word not in vocab_hindi:
        word = '<unk>'
      wordvec_hindi.append(vocab_hindi[word])
    item['trg'] = wordvec_hindi
  return data

In [0]:
vectorized_data = vectorization(training_data)

In [0]:
type(vectorized_data)

list

In [0]:
vectorized_data[:2]

[{'src': [12, 11, 10, 9, 8, 7, 6, 5, 4], 'trg': [4, 5, 6, 7, 8, 9, 10, 11]},
 {'src': [20, 10, 19, 18, 17, 16, 15, 14, 13],
  'trg': [12, 13, 14, 15, 16, 17, 18, 7, 19, 20, 21, 22, 23]}]

In [0]:
valid_vectorized_data = vectorization(valid_data)
testing_vectorized_data = vectorization(testing_data)

In [0]:
batch_size = 64

In [0]:
def batch_iterator(data,batch_size):
  iterator = []
  data.sort(key = lambda x: len(x['src']))
  for i in range(0,len(data),batch_size):
    iterator.append(data[i:i+batch_size])
  return iterator

In [0]:
training_iterator = batch_iterator(vectorized_data, batch_size)

In [0]:
valid_iterator = batch_iterator(valid_vectorized_data, batch_size)
testing_iterator = batch_iterator(testing_vectorized_data, batch_size)

In [0]:
len(training_iterator[0])

64

In [0]:
type(training_iterator[0])

list

In [0]:
def padding(batch_data):
  max_lenght = 0
  for item in batch_data:
    max_lenght = max(max_lenght, len(item['src']), len(item['trg']))
  
  for item in batch_data:
    extra_src = max_lenght - len(item['src'])
    extra_trg = max_lenght - len(item['trg'])
    item['src'] = [0] + item['src'] + [3]*extra_src + [1]
    item['trg'] = [0] + item['trg'] + [3]*extra_trg + [1]
  return batch_data

In [0]:
batched_iterator = []
for batch in training_iterator:
  batched_iterator.append(padding(batch))

In [0]:
valid_padded_iterator = []
for batch in valid_iterator:
  valid_padded_iterator.append(padding(batch))

In [0]:
testing_padded_iterator = []
for batch in testing_iterator:
  testing_padded_iterator.append(padding(batch))

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [0]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

In [0]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [0]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [0]:
INPUT_DIM = len(vocab_english)
OUTPUT_DIM = len(vocab_hindi)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [0]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(13445, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(25598, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=25598, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [0]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 45,937,918 trainable parameters


In [0]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters())

In [0]:
criterion = nn.CrossEntropyLoss(ignore_index = 3).to(device)

In [0]:
# for i, batch in enumerate(batched_iterator):
#   if i==0:
#     #print([item['src'] for item in batch])
#     src = torch.LongTensor([item['src'] for item in batch])
#     #print("src:",src)
#     print(src.shape)
#     trg = torch.LongTensor([item['trg'] for item in batch])
#     #print("trg:",trg)
#     print(trg.shape)
#     #print([item['trg'] for item in batch])

In [0]:
# for i, batch in enumerate(batched_iterator):
#   if i==0:
#     #print([item['src'] for item in batch])
#     #src = torch.LongTensor([item['src'] for item in batch])

#     print([item['src'] for item in batch])

In [0]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = torch.t(torch.LongTensor([item['src'] for item in batch])).to(device)
        trg = torch.t(torch.LongTensor([item['trg'] for item in batch])).to(device)
        
        #print([item['src'] for item in batch])
        #src = torch.LongTensor([item['src'] for item in batch])
        #print((src))
        #trg = torch.LongTensor([item['trg'] for item in batch])
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        #output = output[1:].view(-1, output_dim)
        output = output[1:].contiguous().view(-1, output_dim)
        trg = trg[1:].contiguous().view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = torch.t(torch.LongTensor([item['src'] for item in batch])).to(device)
            trg = torch.t(torch.LongTensor([item['trg'] for item in batch])).to(device)

            #src = torch.LongTensor([item['src'] for item in batch])
            #trg = torch.LongTensor([item['trg'] for item in batch])

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].contiguous().view(-1, output_dim)
            trg = trg[1:].contiguous().view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [0]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [0]:
import time
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, batched_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_padded_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    #print(train_loss)
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 2m 4s
	Train Loss: 6.790 | Train PPL: 888.580
	 Val. Loss: 8.153 |  Val. PPL: 3475.271
Epoch: 02 | Time: 2m 3s
	Train Loss: 5.873 | Train PPL: 355.379
	 Val. Loss: 7.820 |  Val. PPL: 2489.176
Epoch: 03 | Time: 2m 3s
	Train Loss: 5.340 | Train PPL: 208.441
	 Val. Loss: 7.953 |  Val. PPL: 2844.537
Epoch: 04 | Time: 2m 3s
	Train Loss: 4.783 | Train PPL: 119.453
	 Val. Loss: 7.952 |  Val. PPL: 2842.422
Epoch: 05 | Time: 2m 3s
	Train Loss: 4.226 | Train PPL:  68.474
	 Val. Loss: 8.145 |  Val. PPL: 3444.877
Epoch: 06 | Time: 2m 3s
	Train Loss: 3.765 | Train PPL:  43.180
	 Val. Loss: 8.120 |  Val. PPL: 3359.649
Epoch: 07 | Time: 2m 3s
	Train Loss: 3.390 | Train PPL:  29.661
	 Val. Loss: 8.265 |  Val. PPL: 3883.621
Epoch: 08 | Time: 2m 3s
	Train Loss: 3.096 | Train PPL:  22.102
	 Val. Loss: 8.535 |  Val. PPL: 5091.567
Epoch: 09 | Time: 2m 3s
	Train Loss: 2.876 | Train PPL:  17.748
	 Val. Loss: 8.628 |  Val. PPL: 5586.818
Epoch: 10 | Time: 2m 3s
	Train Loss: 2.672 | Train PPL:

In [0]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss = evaluate(model, testing_padded_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 7.762 | Test PPL: 2350.606 |


In [0]:
reverse_vocab_hindi = dict((i,word) for word, i in vocab_hindi.items())
reverse_vocab_english = dict((i,word) for word, i in vocab_english.items())

In [0]:
def translate_sentence(sentence, model, device, max_len = 50):
    
    # model.eval()
    # word2vec = [0]    
    # for word in sentence:
    #   if word not in vocab_english:
    #     word = '<unk>'
    #   word2vec.append(vocab_english[word])
    # word2vec.append(1)

    model.eval()
    word2vec = [1]    
    for word in sentence:
      if word not in vocab_english:
        word = '<unk>'
      word2vec.append(vocab_english[word])
    word2vec.append(0)
    word2vec = word2vec[::-1]

    # word2vec = []
    # for word in sentence:
    #   if word not in vocab_english:
    #     word = '<unk>'
    #   word2vec.append(vocab_english[word])
    # word2vec=word2vec[::-1]
    # word2vec=[0]+word2vec+[1]

    src_tensor = torch.LongTensor(word2vec).unsqueeze(0).to(device)   
    with torch.no_grad():
        hidden, cell = model.encoder(src_tensor)
        #print(hidden, cell)    
    print(src_tensor[0])
    with torch.no_grad():
        prediction, h, c = model.decoder(src_tensor[0],hidden, cell)
    
    #print("prediction",prediction, prediction.shape)

    translation_tensor = torch.argmax(prediction.squeeze(1), 1)
    #print(translation_tensor)
    pred_indexes=translation_tensor.data.tolist()
    output = [reverse_vocab_hindi[i] for i in pred_indexes]
    print(" ".join(output))

In [0]:
source = ['dwarka','is','also','a','dhaam','among','the','chaar','dhaams']
translate_sentence(source,model,device)

ही रंग तो है में एक में लायक बार में झील


In [0]:
source = ["how", "to", "reach", "jammu"]
translate_sentence(source,model,device)

ही के में रूप सकेगा झील


In [0]:
source=["it", "is", "really", "adventurous"]
translate_sentence(source,model,device)

ही के एक बार सकती झील


In [0]:
sentence=["it", "is", "really", "adventurous"]

In [0]:
word2vec = [1]    
for word in sentence:
  if word not in vocab_english:
    word = '<unk>'
  word2vec.append(vocab_english[word])
word2vec.append(0)
word2vec = word2vec[::-1]

# word2vec = []
# for word in sentence:
#   if word not in vocab_english:
#     word = '<unk>'
#   word2vec.append(vocab_english[word])
# word2vec=word2vec[::-1]
# word2vec=[0]+word2vec+[1]

src_tensor = torch.LongTensor(word2vec).unsqueeze(0).to(device)   
print(src_tensor)
with torch.no_grad():
    hidden, cell = model.encoder(src_tensor)
    #print(hidden, cell)    
print(src_tensor[0])

with torch.no_grad():
    prediction, h, c = model.decoder(src_tensor[0],hidden, cell)

#print("prediction",prediction, prediction.shape)

translation_tensor = torch.argmax(prediction.squeeze(1), 1)
print(translation_tensor)
pred_indexes=translation_tensor.data.tolist()


tensor([[   0,  979, 3664,    5,   88,    1]], device='cuda:0')
tensor([   0,  979, 3664,    5,   88,    1], device='cuda:0')
tensor([ 162,   17,    8,   86, 2375, 2398], device='cuda:0')
